In [73]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [74]:
import sys; sys.prefix
#check where Im installing these packages & where I am looking for them

'C:\\ProgramData\\anaconda3'

In [131]:
# all declare degrees data
df = pd.read_csv("Business Major Declaration_June 2, 2023_14.21.csv", low_memory=False)
df=  df[df.Finished == True]

#Split Qualtrics data responses into DECLARES and DROPS datasets
dec =df.filter(regex=("DECLARE.*"))

#making unified column major names
for col in dec.columns:
    dec.rename(columns={col : ("DECLARE" + col.split("Center. -",1)[1])}, inplace = True)
# rejoin with 2 other columns we want from original dataset
dec = df[["Recorded Date", "Please provide your information below. - Campus ID", "Location Latitude", "Location Longitude"]].join(dec)
dec.columns = dec.columns.str.replace("Please provide your information below. - Campus ID", "Campus ID")

# to long format, grouping by campus id & recorded date
m_dec = dec.melt(id_vars =['Campus ID', 'Recorded Date', "Location Latitude", "Location Longitude"], value_vars = dec.columns.tolist() )
pd.set_option('display.float_format', lambda x: f'{x:.0f}')
m_dec = m_dec.dropna()
m_dec

C:\Users\kmconrad3\AppData\Local\Temp\ipykernel_13192\2222442171.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  dec.columns = dec.columns.str.replace("Please provide your information below. - Campus ID", "Campus ID")


,Campus ID,Recorded Date,Location Latitude,Location Longitude,variable,value
4,9068197657,12/29/2014 22:51,43,-88,DECLARE Accounting (BUS 004),Accounting (BUS 004)
6,9065702640,1/6/2015 19:16,45,-93,DECLARE Accounting (BUS 004),Accounting (BUS 004)
7,9068724435,1/8/2015 11:08,43,-88,DECLARE Accounting (BUS 004),Accounting (BUS 004)
8,9068370601,1/9/2015 8:51,43,-89,DECLARE Accounting (BUS 004),Accounting (BUS 004)
9,9065976061,1/9/2015 14:19,41,-73,DECLARE Accounting (BUS 004),Accounting (BUS 004)
...,...,...,...,...,...,...
204779,9081311251,3/30/2022 13:07,44,11,DECLARE Summer Certificate in Business Fundame...,Summer Certificate in Business Fundamentals fo...
204787,9081224397,3/30/2022 17:29,43,-89,DECLARE Summer Certificate in Business Fundame...,Summer Certificate in Business Fundamentals fo...
204794,9079188588,3/31/2022 13:09,43,-89,DECLARE Summer Certificate in Business Fundame...,Summer Certificate in Business Fundamentals fo...
204802,9082172322,4/2/2022 12:32,42,-88,DECLARE Summer Certificate in Business Fundame...,Summer Certificate in Business Fundamentals fo...


In [76]:
drop = df.filter(regex=("CANCEL.*"))
for col in drop.columns:
    drop.rename(columns={col : ("CANCEL" + col.split("CANCEL. -",1)[1])}, inplace = True)
drop = df[["Recorded Date", "Please provide your information below. - Campus ID", "Location Latitude", "Location Longitude"]].join(drop) 
drop.columns = drop.columns.str.replace("Please provide your information below. - Campus ID", "Campus ID")

#p_drop = pd.wide_to_long(drop, stubnames="CANCEL", i=['Recorded Date', 'Campus ID'], j="Majors names", suffix="\\D+")
#p_drop

m_drop = drop.melt(id_vars =['Campus ID', 'Recorded Date', "Location Latitude", "Location Longitude"], value_vars = drop.columns.tolist() )
pd.set_option('display.float_format', lambda x: f'{x:.0f}')
m_drop = m_drop.dropna()

# EX. check on 9064111017
#m_drop[ m_drop["Campus ID"] == 9067583048].dropna()

C:\Users\kmconrad3\AppData\Local\Temp\ipykernel_13192\2278109724.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  drop.columns = drop.columns.str.replace("Please provide your information below. - Campus ID", "Campus ID")


In [77]:
#COMBINED DEC & DROP 
dec_drop = m_dec.append(m_drop)

# renaming columns
dec_drop = dec_drop.rename(columns= {"variable" : "Declare/Drop", "value" : "Major name"})
dec_drop["Declare/Drop"] = dec_drop["Declare/Drop"].apply(lambda x: "DECLARE" if "DECLARE" in x else "CANCEL")
dec_drop

#EX. dec_drop[dec_drop["Campus ID"]==9069621325]

C:\Users\kmconrad3\AppData\Local\Temp\ipykernel_13192\1475627556.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dec_drop = m_dec.append(m_drop)


,Campus ID,Recorded Date,Location Latitude,Location Longitude,Declare/Drop,Major name
4,9068197657,12/29/2014 22:51,43,-88,DECLARE,Accounting (BUS 004)
6,9065702640,1/6/2015 19:16,45,-93,DECLARE,Accounting (BUS 004)
7,9068724435,1/8/2015 11:08,43,-88,DECLARE,Accounting (BUS 004)
8,9068370601,1/9/2015 8:51,43,-89,DECLARE,Accounting (BUS 004)
9,9065976061,1/9/2015 14:19,41,-73,DECLARE,Accounting (BUS 004)
...,...,...,...,...,...,...
236723,9082174112,1/7/2022 14:49,34,-119,CANCEL,Summer Certificate in Business Fundamentals fo...
236753,9079555299,1/17/2022 20:13,34,-119,CANCEL,Summer Certificate in Business Fundamentals fo...
236775,9077191758,1/21/2022 13:54,41,-74,CANCEL,Summer Certificate in Business Fundamentals fo...
236786,9082058885,1/24/2022 12:34,43,-89,CANCEL,Summer Certificate in Business Fundamentals fo...


In [78]:
# create Term column

dec_drop["Term"] = dec_drop["Recorded Date"].apply(lambda x: "Spring  {}".format(x.split("/")[2][:4]) if int(x.split("/")[0])<6 \
                                       else ("Summer {}".format(x.split("/")[2][:4]) if (int(x.split("/")[0])>=6 and int(x.split("/")[0])<9) \
                                           else "Fall {}".format(x.split("/")[2][:4]) ) )

# create accademic year column
dec_drop["Academic Year"] = dec_drop["Recorded Date"].apply(lambda x: "{}-{}".format(x.split("/")[2][:4], int(x.split("/")[2][:4])+1) if (int(x.split("/")[0])>=6 and int(x.split("/")[0])<9)
                                                            else ( "{}-{}".format(x.split("/")[2][:4], int(x.split("/")[2][:4])+1) if int(x.split("/")[0])>=9 \
                                                                  else "{}-{}".format(int(x.split("/")[2][:4])-1, x.split("/")[2][:4])) ) 



# EX. dec_drop[dec_drop["Campus ID"]==9069621325]
#dec_drop.groupby("Campus ID")
#dec_drop

In [79]:
# COORDINATES TO ZIPS
!python3 -m install arcgis
from arcgis.geocoding import reverse_geocode
#from arcgis.geometry import Geometry
#from arcgis.gis import GIS
#!conda install -c esri arcgis
#!conda list
!pip install arcgis  # gives "Defaulting to user installation because normal site-packages is not writeable" notice

from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS
gis = GIS("http://www.arcgis.com", "***", "***")
def get_zip(df, lon_field, lat_field):
    location = reverse_geocode((Geometry({"x":float(df[lon_field]), "y":float(df[lat_field]), "spatialReference":{"wkid": 4326}})))
    return location['address']['Postal']
zipps = dec_drop.apply(get_zip, axis=1,  lat_field="Location Latitude", lon_field="Location Longitude")
zipps



#!conda install -c conda-forge geopy
#!pip freeze > requirements.txt
#!pip install -r requirements.txt
!python3 -m pip install geopy
!pip install geopy # gives "Defaulting to user installation because normal site-packages is not writeable" notice

#import geopy
#def get_zipcode(df, geolocator, lat_field, lon_field):
   # location = geolocator.reverse((df[lat_field], df[lon_field]))
   # return location.raw['address']['postcode']
#geolocator = geopy.Nominatim(user_agent='my-application')
#zipps = dec_drop.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field="Location Latitude", lon_field="Location Longitude")

C:\Users\kmconrad3\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe: No module named install


ModuleNotFoundError: No module named 'arcgis'

In [82]:
#export data 
dec_drop.to_excel("output_formatting.xlsx", "output_formatting.xlsx")

# Merging original data to current form

In [229]:
df2 = pd.read_csv("Major+Dec+V2_June+14,+2023_13.41.csv")

In [199]:
df2.columns = df2.iloc[0]
df2 = df2[2:]
#df2=  df2[df2.Finished == True]

In [220]:
#Declare
dec2 =df2.filter(regex=("DECLARE.*"))

#making unified column major names
for col in dec2.columns:
    dec2.rename(columns={col : ("DECLARE" + col.split("Center. -",1)[1])}, inplace = True)
#rejoin with 2 other columns we want from original dataset
dec2 = df2[["Recorded Date", "Please provide your information below. - Campus ID", "Location Latitude", "Location Longitude"]].join(dec2)
dec2.columns = dec2.columns.str.replace("Please provide your information below. - Campus ID", "Campus ID")


# to long format, grouping by campus id & recorded date
m_dec2 = dec2.melt(id_vars =['Campus ID', 'Recorded Date', "Location Latitude", "Location Longitude"], value_vars = dec2.columns.tolist(), var_name='variable')
pd.set_option('display.float_format', lambda x: f'{x:.0f}')
m_dec2 = m_dec2.dropna()

C:\Users\kmconrad3\AppData\Local\Temp\ipykernel_13192\965901724.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  dec2.columns = dec2.columns.str.replace("Please provide your information below. - Campus ID", "Campus ID")


In [218]:
#Drops
drop2 =df2.filter(regex=("CANCEL.*"))

#making unified column major names
for col in drop2.columns:
    drop2.rename(columns={col : ("CANCEL" + col.split("CANCEL. -",1)[1])}, inplace = True)
#rejoin with 2 other columns we want from original dataset
drop2 = df2[["Recorded Date", "Please provide your information below. - Campus ID", "Location Latitude", "Location Longitude"]].join(drop2)
drop2.columns = drop2.columns.str.replace("Please provide your information below. - Campus ID", "Campus ID")


# to long format, grouping by campus id & recorded date
m_drop2 = drop2.melt(id_vars =['Campus ID', 'Recorded Date', "Location Latitude", "Location Longitude"], value_vars = drop2.columns.tolist(),  var_name='variable')
pd.set_option('display.float_format', lambda x: f'{x:.0f}')
m_drop2 = m_drop2.dropna()
m_drop2

C:\Users\kmconrad3\AppData\Local\Temp\ipykernel_13192\2224225640.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  drop2.columns = drop2.columns.str.replace("Please provide your information below. - Campus ID", "Campus ID")


,Campus ID,Recorded Date,Location Latitude,Location Longitude,variable,value
14,1234,2022-04-01 12:35:59,43.0724945068359375,-89.4490966796875,CANCEL Accounting (BUS 004),Accounting (BUS 004)
50,9082962946,2022-04-06 12:58:07,43.0812000000000026034285838250070810317993164...,-89.3845000000000027284841053187847137451171875,CANCEL Accounting (BUS 004),Accounting (BUS 004)
87,9082335846,2022-04-11 14:31:08,43.081207275390625,-89.38449859619140625,CANCEL Accounting (BUS 004),Accounting (BUS 004)
96,9083640012,2022-04-11 16:49:38,43.081207275390625,-89.38449859619140625,CANCEL Accounting (BUS 004),Accounting (BUS 004)
130,9082633554,2022-04-14 09:37:43,43.0812,-89.3845,CANCEL Accounting (BUS 004),Accounting (BUS 004)
...,...,...,...,...,...,...
75860,9082902017,2023-04-12 11:29:32,42.0128,-88.0967,CANCEL Consulting Certificate (CERT 130),Consulting Certificate (CERT 130)
75977,9083171273,2023-04-17 11:00:33,43.0782,-89.4075,CANCEL Consulting Certificate (CERT 130),Consulting Certificate (CERT 130)
76113,9083003906,2023-04-21 10:06:13,43.0684,-89.4293,CANCEL Consulting Certificate (CERT 130),Consulting Certificate (CERT 130)
76130,9080763411,2023-04-22 14:20:24,43.0684,-89.4293,CANCEL Consulting Certificate (CERT 130),Consulting Certificate (CERT 130)


In [226]:
dec_drop2 = m_dec2.append(m_drop2)
dec_drop2.columns

# renaming columns

dec_drop2 = dec_drop2.rename(columns= {"variable" : "Declare/Drop"})
dec_drop2 = dec_drop2.rename(columns= { "value" : "Major name"})
dec_drop2["Declare/Drop"] = dec_drop2["Declare/Drop"].apply(lambda x: "DECLARE" if "DECLARE" in x else "CANCEL")

dec_drop2["Term"] = dec_drop2["Recorded Date"].apply(lambda x: "Spring  {}".format(x.split("-")[0][:4]) if int(x.split("-")[1])<6 \
                                       else ("Summer {}".format(x.split("-")[0][:4]) if (int(x.split("-")[0])>=6 and int(x.split("-")[1])<9) \
                                           else "Fall {}".format(x.split("-")[0][:4]) ) )

# create accademic year column
dec_drop2["Academic Year"] = dec_drop2["Recorded Date"].apply(lambda x: "{}-{}".format(x.split("-")[0][:4], int(x.split("-")[0][:4])+1) if (int(x.split("-")[1])>=6 and int(x.split("-")[1])<9)
                                                            else ( "{}-{}".format(x.split("-")[0][:4], int(x.split("-")[0][:4])+1) if int(x.split("-")[1])>=9 \
                                                                  else "{}-{}".format(int(x.split("-")[0][:4])-1, x.split("-")[0][:4])) ) 



dec_drop2 = dec_drop.append(dec_drop2)

C:\Users\kmconrad3\AppData\Local\Temp\ipykernel_13192\2850734559.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dec_drop2 = m_dec2.append(m_drop2)
C:\Users\kmconrad3\AppData\Local\Temp\ipykernel_13192\2850734559.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dec_drop2 = dec_drop.append(dec_drop2)


In [228]:
#ALL DATA
dec_drop2.to_excel("output_formatting.xlsx", "output_formatting.xlsx")